In [ ]:
#Test parameters D and a effect on wave propagation

import numpy as np                                                                                                                 
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#Define functions used for diffusion partial differential equations

#Define Laplace operator with zero-flux boundary condition
def dd2 (A,x,y,dx,dy,i,j):
  if i==0:
    dAx2=(A[i+1,j]-2*A[i,j]+A[i+1,j])/(dx*dx)
  elif i==x-1:
    dAx2=(A[i-1,j]-2*A[i,j]+A[i-1,j])/(dx*dx)
  else:
    dAx2=(A[i+1,j]-2*A[i,j]+A[i-1,j])/(dx*dx)

  if j==0:
    dAy2=(A[i,j+1]-2*A[i,j]+A[i,j+1])/(dy*dy)
  elif j==y-1:
    dAy2=(A[i,j-1]-2*A[i,j]+A[i,j-1])/(dy*dy)
  else:
    dAy2=(A[i,j+1]-2*A[i,j]+A[i,j-1])/(dy*dy)
    
  d=dAx2+dAy2

  return d;

#Define Erk propagation function
#E0, initial condition
#Cell id, cells position
#T, total simulation time
#dt, simulation time step
#c_threhold, activition threshold
#cell_len, cell length
#activition rate, a
#Diffusion coefficient, D

def Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,cell_len,a,D):  
   t1=0
   t2=0
   E=E0.copy()
   v=0
   L=np.zeros((step-1,num_grid_x))
   for time in range(step-1): 
     Es=E
     for i in range(num_grid_x):
       for j in range(num_grid_y):
         if i<10:
           E[i,j]=Es[i,j]+dt*(D*dd2(Es,num_grid_x,num_grid_y,grid_size,grid_size,i,j)+a*(1-Es[i,j]))
         else:
           E[i,j]=Es[i,j]+dt*(D*dd2(Es,num_grid_x,num_grid_y,grid_size,grid_size,i,j)+a*theta_n(Es,Cell_id,i,j,c_threhold,cell_len))
     if t1==0:
       if np.min(E[11,:])>0.1:
         t1=time
     if t2==0:
       if np.min(E[num_grid_x-5,:])>0.1:
         t2=time
         v=((num_grid_x-16)*cell_len*grid_size)/((t2-t1)*dt)
     else:
        break
     L[time,:]=np.mean(E, axis=1).T
       
   return E,v,L;

#Define Heaviside step function
def theta_n (E,Cell_id,i,j,c_threhold,cell_len): 
  c=0
  if Cell_id[i,j]<0:
    print('cell identity error')    
  if Cell_id[i,j]==0:
    c=0
  else:
    if max(E[int(i-Cell_id[i,j]+1):min(int(i-Cell_id[i,j]+1+cell_len),num_grid_x),j])>c_threhold:
        c=1-E[i,j]
  return c;

#Generate cell pattern for signal propagation with given parameters
#num_grid_x, num_grid_y, number of cells on x- and y-axis
#bias, distribution bias of cells
#den_volum, volume density of long cells
#cell_len, long cell length

def Generate_cell_id (num_grid_x,num_grid_y,bias,den_volum,cell_len): 
  Cell_id=np.zeros((num_grid_x,num_grid_y))-1
  dev_h=den_volum+bias
  dev_l=den_volum-bias
  density_h=dev_h/(cell_len+dev_h-cell_len*dev_h)
  density_l=dev_l/(cell_len+dev_l-cell_len*dev_l)
  for j in range(num_grid_y):
    i=0
    while i<num_grid_x:
      indic=np.random.uniform()
      if j<0.5*num_grid_y:
        den= density_h
      else:
        den= density_l
      if indic>den:
        Cell_id[i,j]=0
        i=i+1
      else:
        d=min(cell_len,num_grid_x-i)
        Cell_id[i:i+d,j]=np.linspace(1,d,num=d)
        i=i+cell_len

  return Cell_id;

In [ ]:
#Simulation parameters set up

T=500 #min

range_grid_x=1000
range_grid_y=200
grid_size=10
num_grid_x=int(range_grid_x/grid_size)
num_grid_y=int(range_grid_y/grid_size)
E0=np.zeros((num_grid_x,num_grid_y))

dt=0.01
step=int(T/dt)

den=0.9
bias=0

In [ ]:
#D and a varing range
scale=np.array([2/5, 3/5, 4/5, 5/5, 6/5, 7/5, 8/5, 9/5, 10/5, 11/5])
scale=scale*scale

D=0.1*60
c_threhold=0.1
a=0.2

Dr=D*scale
ar=a*scale

In [ ]:
#Simulation

c_len=4 
#or 6 and 8 to general plots in Figure S4H
repeat=1

data=np.zeros((repeat,len(scale)))
for r in range(repeat):
    print(r)
    for i in range(len(Dr)):
        E0=np.zeros((num_grid_x,num_grid_y))
        Cell_id=Generate_cell_id (num_grid_x,num_grid_y,bias,den,c_len)
        E,v,L2 = Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,c_len,a,Dr[i])
        data[r,i]=v
        
np.savetxt('erk_simulation_model_D.csv', data, delimiter=',')

data=np.zeros((repeat,len(scale)))
for r in range(repeat):
    print(r)
    for i in range(len(ar)):
        E0=np.zeros((num_grid_x,num_grid_y))
        Cell_id=Generate_cell_id (num_grid_x,num_grid_y,bias,den,c_len)
        E,v,L2 = Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,c_len,ar[i],D)
        data[r,i]=v

np.savetxt('erk_simulation_model_a.csv', data, delimiter=',')

In [ ]:
#plotting results

data_D = np.loadtxt('erk_simulation_model_D.csv', delimiter=',')
data_a = np.loadtxt('erk_simulation_model_a.csv', delimiter=',')

c1='#EF6145'
c2='#389ADD'
c3='#45A39B'

x=np.array([2/5, 3/5, 4/5, 5/5, 6/5, 7/5, 8/5, 9/5, 10/5, 11/5])
x=x*x
plt.figure(figsize=(4, 3.5))

plt.scatter(x,data_D*60,color=c1,s=20,alpha=0.7)
plt.scatter(x,data_a*60,color=c2,s=20,alpha=0.7)